In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
import os
from tensorflow.keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator,load_img, img_to_array
from tensorflow.keras.metrics import categorical_crossentropy
from keras.models import Sequential, Model
from keras.layers import Conv2D, MaxPooling2D,GlobalAveragePooling2D
from keras.layers import Activation, Dropout, BatchNormalization, Flatten, Dense, AvgPool2D,MaxPool2D
from keras.models import Sequential, Model
from keras.applications.vgg16 import VGG16, preprocess_input
from keras.optimizers import Adam, SGD, RMSprop
from keras import backend as K
import cv2
import glob

In [ ]:
!pip install -U efficientnet

In [ ]:
from pathlib import Path
train_path = Path("../input/sports-classification/train")
train_path.glob(r'**/*.jpg')

valid_path = Path("../input/sports-classification/valid")
valid_path.glob(r'**/*.jpg')

test_path = Path("../input/sports-classification/test")
test_path.glob(r'**/*.jpg')

In [ ]:
train_filepath = list(train_path.glob(r'**/*.jpg'))
valid_filepath = list(valid_path.glob(r'**/*.jpg'))
test_filepath = list(test_path.glob(r'**/*.jpg'))

In [ ]:
os.path.split(os.path.split(train_filepath[0])[1]) 

In [ ]:
len(list(map(lambda x: os.path.split(os.path.split(x)[0])[1],train_filepath)))

In [ ]:
classes_train = list(map(lambda x: os.path.split(os.path.split(x)[0])[1],train_filepath))
classes_valid = list(map(lambda x: os.path.split(os.path.split(x)[0])[1],valid_filepath))
classes_test = list(map(lambda x: os.path.split(os.path.split(x)[0])[1],test_filepath))

In [ ]:
train_df = pd.concat([pd.Series(train_filepath).astype(str), pd.Series(classes_train)], axis=1)
train_df.columns = ['Images', 'Image_label']

test_df = pd.concat([pd.Series(test_filepath).astype(str), pd.Series(classes_test)], axis=1)
test_df.columns = ['Images', 'Image_label']

valid_df = pd.concat([pd.Series(valid_filepath).astype(str), pd.Series(classes_valid)], axis=1)
valid_df.columns = ['Images', 'Image_label']


In [ ]:
test_df.shape

In [ ]:

train_df.shape

In [ ]:
valid_df.shape

In [ ]:
train_datagen = ImageDataGenerator(rescale = 1./255.,rotation_range = 40, width_shift_range = 0.2, height_shift_range = 0.2, 
                                   shear_range = 0.2, zoom_range = 0.2, horizontal_flip = True, vertical_flip =True)
test_datagen = ImageDataGenerator(rescale = 1.0/255.)

In [ ]:
train_gen = train_datagen.flow_from_dataframe(dataframe = train_df,
                                              x_col = 'Images', y_col ='Image_label',
                                              target_size = (200,200), batch_size = 512, 
                                              class_mode = 'categorical', shuffle = True)
val_gen = train_datagen.flow_from_dataframe(valid_df, 
                                            target_size=(200,200), x_col = 'Images', y_col ='Image_label', 
                                            class_mode='categorical',
                                            batch_size= 512, shuffle=True)
test_gen = test_datagen.flow_from_dataframe(test_df,
                                            target_size = (200,200), x_col = 'Images', y_col ='Image_label',
                                             class_mode = 'categorical',
                                            batch_size = 512, shuffle = False)

In [ ]:
train_gen.class_indices

In [ ]:
len(train_gen.class_indices)

In [ ]:

import efficientnet.keras as efn

base_model =  efn.EfficientNetB0(input_shape = (224, 224, 3), # Shape of our images
include_top = False, # Leave out the last fully connected layer
weights = 'imagenet')

for layer in base_model.layers:
    layer.trainable = False
    
base_model.summary()  

In [ ]:
model = Sequential()
model.add(base_model)
model.add(GlobalAveragePooling2D())
model.add(Dense(128))
model.add(Dropout(0.2))
model.add(Dense(73, activation = 'softmax'))
model.summary()

In [ ]:
callbacks = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=2)
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
history = model.fit(train_gen, validation_data = val_gen, epochs = 10, 
                    callbacks = [callbacks], verbose = 1)

In [ ]:
print("training_accuracy", history.history['accuracy'][-1])
print("validation_accuracy", history.history['val_accuracy'][-1])

In [ ]:
predictions = model.predict(test_gen)
preds = np.argmax(predictions, axis=1)
preds

In [ ]:
model.evaluate(test_gen)

In [ ]:
labels = train_gen.class_indices
labels = dict((v,k) for k,v in labels.items())
preds = [labels[k] for k in preds]
preds

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report
matrix = confusion_matrix(test_df.Image_label , preds)

matrix

In [ ]:
print(classification_report(test_df.Image_label, preds))

In [ ]:
accuracy_score(test_df.Image_label, preds)

In [ ]:
model.evaluate(test_gen)